# Set up


## Imports

In [3]:
%matplotlib inline

In [4]:
import sys
import os
from pathlib import Path
import math

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context="talk", font_scale=1, rc=None)
sns.set_style('whitegrid')

In [6]:
# Load the "autoreload" extension
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
from pandas import IndexSlice as idx
import numpy as np

from collections import OrderedDict

from munch import Munch
from tqdm import tqdm

In [8]:
import engarde.decorators as ed
import pyparsing as p

In [9]:
# import crunchers.sklearn_helpers.exploration as expl
# import crunchers.statsmodels_helpers.lazy_stats as stats
# import crunchers.pandas_helpers.transformations as xform

In [10]:
import biorep_etl.data.hreg_load_recode as loading
import biorep_etl.data.field_definitions.hreg_redcap_dump as rcd

In [69]:
x = "([ibdtype] = '0' or [newidbdiag] = '0') and [ibdcondl(6)] = ''"
y = """[ibdcondl(6)] = '1'"""
z= "[meddraae25] = '99' or [meddraae24] = '99' or [meddraae23] = '99' or [meddraae22] = '99' or [meddraae21] = '99' or [meddraae20] = '99' or [meddraae19] = '99' or [meddraae18] = '99' or [meddraae17] = '99' or [meddraae166] = '99' or [meddraae15] = '99' or [meddraae14] = '99' or [meddraae13] = '99' or [meddraae12] = '99' or [meddraae11] = '99' or [meddraae10] = '99' or [meddraae9] = '99' or [meddraae8] = '99' or [meddraae7] = '99' or [meddraae6] = '99' or [meddraae5] = '99' or [meddraae4] = '99' or [meddraae3] = '99' or [meddraae2] = '99' or [meddraae1] = '99' or [meddraae26] = '99' or [meddraae27] = '99' or [meddraae28] = '99' or [meddraae29] = '99' or [meddraae16] = '99' "

In [76]:
# misc
lbrack = p.Literal("[").suppress()
rbrack = p.Literal("]").suppress()
lparen = p.Literal("(").suppress()
rparen = p.Literal(")").suppress()
semi = p.Literal(";")
squot = p.Literal("'")#.suppress()
dquot = p.Literal('"')#.suppress()
empty_quotes = p.Literal("''") | p.Literal('""')
anyquote = p.oneOf("""" '""").suppress()
anyquote_opt = p.Optional(anyquote)


# variables
checkbox_val = p.Optional('('+p.Word(p.nums)+')')
variable = p.Combine(lbrack+p.Word(p.alphanums + "_")+checkbox_val+rbrack)


# operators
operator = p.oneOf("<> >= <= > < =")


# logic gates
AND = p.Literal("and")
OR = p.Literal("or")
logic_gate = AND | OR



# comparison vals
comparison_val = p.Combine(anyquote_opt + p.Word(p.alphanums, excludeChars='[ ]') + anyquote_opt) | empty_quotes


# branch expression
branch_expression = variable + operator + comparison_val


# conditional expression
conditional_exp = branch_expression + logic_gate + branch_expression

# nested conditional
nested_conditional_exp = p.Optional(lparen) + conditional_exp + p.Optional(rparen + logic_gate + branch_expression)

# branch_logic
branch_logic =   branch_expression | conditional_exp | nested_conditional_exp


In [77]:
list(comparison_val.scanString(x))

[((['ibdtype'], {}), 2, 9),
 ((['0'], {}), 13, 16),
 ((['or'], {}), 17, 19),
 ((['newidbdiag'], {}), 21, 31),
 ((['0'], {}), 35, 38),
 ((['and'], {}), 40, 43),
 ((['ibdcondl'], {}), 45, 53),
 ((['6'], {}), 54, 55),
 ((["''"], {}), 60, 62)]

In [67]:


def branch_parser():
    # misc
    lbrack = p.Literal("[").suppress()
    rbrack = p.Literal("]").suppress()
    lparen = p.Literal("(").suppress()
    rparen = p.Literal(")").suppress()
    anyquote = p.oneOf("""" '""").suppress()
    anyquote_opt = p.Optional(anyquote)
    empty_quotes = p.Literal("''") | p.Literal('""')

    # variables
    checkbox_val = p.Optional('('+p.Word(p.nums)+')')
    variable = p.Combine(lbrack+p.Word(p.alphanums + "_")+checkbox_val+rbrack)

    # operators
    operator = p.oneOf("<> >= <= > < =")

    # logic gates
    AND = p.Literal("and")
    OR = p.Literal("or")
    logic_gate = AND | OR
    
    # comparison vals
    comparison_val = (anyquote + p.Word(p.alphanums, excludeChars='[]') + anyquote) | empty_quotes
    
    atom = variable | comparison_val
    
    expr = p.operatorPrecedence( atom,
                                [(operator, 2, p.opAssoc.LEFT),
                                 (logic_gate, 2, p.opAssoc.RIGHT),]
                                )
    return expr

    
    
# expr = p.operatorPrecedence( operand,
#     [("!", 1, p.opAssoc.LEFT),
#      ("^", 2, p.opAssoc.RIGHT),
#      (signop, 1, p.opAssoc.RIGHT),
#      (multop, 2, p.opAssoc.LEFT),
#      (plusop, 2, p.opAssoc.LEFT),]
#     )
    
def Syntax():
    op = p.oneOf( '+ - / *')
    lpar  = p.Literal( '(' ).suppress()
    rpar  = p.Literal( ')' ).suppress()
    num = p.Word(p.nums)
    expr = p.Forward()
    atom = num | p.Group( lpar + expr + rpar )
    expr << atom + p.ZeroOrMore( op + expr )
    return expr    

def test(s):
    results = expr.parseString( s )
    return results

expr = Syntax()

In [48]:
branch = branch_parser()

In [49]:
x

"([ibdtype] = '0' or [newidbdiag] = '0') and [ibdcondl(6)] = ''"

In [59]:
print(branch.parseString(x)[0][2])

ParseException: Expected {{Suppress:(" | ') W:(ABCD...) Suppress:(" | ')} | "''" | """"} (at char 0), (line:1, col:1)

In [65]:
list(comparison_val.scanString(x))

[((['ibdtype'], {}), 2, 9),
 ((['0'], {}), 13, 16),
 ((['or'], {}), 17, 20),
 ((['newidbdiag'], {}), 21, 31),
 ((['0'], {}), 35, 38),
 ((['and'], {}), 40, 44),
 ((['ibdcondl'], {}), 45, 53),
 ((['6'], {}), 54, 55),
 ((["''"], {}), 60, 62)]

# Example

In [ ]:
p.Forward()

In [12]:
#
# simpleArith.py
#
# Example of defining an arithmetic expression parser using
# the operatorGrammar helper method in pyparsing.
#
# Copyright 2006, by Paul McGuire
#

integer = p.Word(p.nums).setParseAction(lambda t:int(t[0]))
variable = p.Word(p.alphas,exact=1)
operand = integer | variable

expop = p.Literal('^')
signop = p.oneOf('+ -')
multop = p.oneOf('* /')
plusop = p.oneOf('+ -')
factop = p.Literal('!')

# To use the operatorGrammar helper:
#   1.  Define the "atom" operand term of the grammar.
#       For this simple grammar, the smallest operand is either
#       and integer or a variable.  This will be the first argument
#       to the operatorGrammar method.
#   2.  Define a list of tuples for each level of operator
#       precendence.  Each tuple is of the form
#       (opExpr, numTerms, rightLeftAssoc, parseAction), where
#       - opExpr is the pyparsing expression for the operator;
#          may also be a string, which will be converted to a Literal
#       - numTerms is the number of terms for this operator (must
#          be 1 or 2)
#       - rightLeftAssoc is the indicator whether the operator is
#          right or left associative, using the pyparsing-defined
#          constants opAssoc.RIGHT and opAssoc.LEFT.
#       - parseAction is the parse action to be associated with 
#          expressions matching this operator expression (the
#          parse action tuple member may be omitted)
#   3.  Call operatorGrammar passing the operand expression and
#       the operator precedence list, and save the returned value
#       as the generated pyparsing expression.  You can then use
#       this expression to parse input strings, or incorporate it
#       into a larger, more complex grammar.
#       
expr = p.operatorPrecedence( operand,
    [("!", 1, p.opAssoc.LEFT),
     ("^", 2, p.opAssoc.RIGHT),
     (signop, 1, p.opAssoc.RIGHT),
     (multop, 2, p.opAssoc.LEFT),
     (plusop, 2, p.opAssoc.LEFT),]
                           )

test = ["9 + 2 + 3",
        "9 + 2 * 3",
        "(9 + 2) * 3",
        "(9 + -2) * 3",
        "(9 + -2) * 3^2^2",
        "(9! + -2) * 3^2^2",
        "M*X + B",
        "M*(X + B)",
        "1+2*-3^4*5+-+-6",]
for t in test:
    print(t)
    print(expr.parseString(t))
    print()


9 + 2 + 3
[[9, '+', 2, '+', 3]]

9 + 2 * 3
[[9, '+', [2, '*', 3]]]

(9 + 2) * 3
[[[9, '+', 2], '*', 3]]

(9 + -2) * 3
[[[9, '+', ['-', 2]], '*', 3]]

(9 + -2) * 3^2^2
[[[9, '+', ['-', 2]], '*', [3, '^', [2, '^', 2]]]]

(9! + -2) * 3^2^2
[[[[9, '!'], '+', ['-', 2]], '*', [3, '^', [2, '^', 2]]]]

M*X + B
[[['M', '*', 'X'], '+', 'B']]

M*(X + B)
[['M', '*', ['X', '+', 'B']]]

1+2*-3^4*5+-+-6
[[1, '+', [2, '*', ['-', [3, '^', 4]], '*', 5], '+', ['-', ['+', ['-', 6]]]]]



# Convert to HTML report and move to reports folder

In [ ]:
# rv.reprt_dir

In [ ]:
# !jupyter nbconvert --to html_toc $rv.ipnb

In [ ]:
# !mkdir -p $rv.reprt_dir
## !mv -f $rv.html $rv.files $rv.reprt_dir